In [2]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import re

In [3]:
def getNames(url):
    df = pd.read_html(requests.get(url).text)[2].loc[:, :'전일거래량']
    df = df[df.종목명.isna() == False]
    return [i for i in df.종목명]

In [4]:
def getIndustryData():
    url = 'https://finance.naver.com/sise/sise_group.nhn?type=upjong'
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    업종 = pd.read_html(html)[0].dropna().reset_index(drop=True)
    업종.columns = [i[1] for i in 업종.columns]
    업종링크 = {}
    for index, link in enumerate(soup.findAll("a", href=re.compile("^(/sise/sise_group_detail.nhn)((?!:).)*$"))):
        if 'href' in link.attrs:
            업종링크[업종.업종명[index]] = 'https://finance.naver.com' + link.attrs['href']
    업종['전일비'] = 업종.전일대비.apply(lambda x: float(x.replace('%', '')))
    업종['링크'] = 업종.업종명.apply(lambda x: 업종링크.get(x))
    업종['대표종목'] = 업종.링크.apply(lambda x: getNames(x))
    업종.sort_values('전일비', ascending=False, inplace=True)
    return pd.concat([업종.head(10), 업종.tail(10)])

In [5]:
getIndustryData()

,업종명,전일대비,전체,상승,보합,하락,등락그래프,전일비,링크,대표종목
0,전기제품,+3.82%,40.0,24.0,4.0,12.0,40%,3.82,https://finance.naver.com/sise/sise_group_deta...,"[디피씨, 에코프로비엠 *, 명성티엔에스 *, 신흥에스이씨 *, 에스퓨얼셀 *, 씨..."
1,건축제품,+3.00%,18.0,10.0,0.0,8.0,32%,3.00,https://finance.naver.com/sise/sise_group_deta...,"[남선알미늄, 국영지앤엠 *, 남선알미우, 다스코, 덕신하우징 *, 정산애강 *, ..."
2,통신장비,+2.61%,48.0,26.0,5.0,17.0,27%,2.61,https://finance.naver.com/sise/sise_group_deta...,"[유비쿼스홀딩스 *, 에치에프알 *, 케이엠더블유 *, 서진시스템 *, 오이솔루션 ..."
3,운송인프라,+2.28%,3.0,1.0,0.0,2.0,24%,2.28,https://finance.naver.com/sise/sise_group_deta...,"[한국공항, 토탈소프트 *, 선광 *]"
4,출판,+1.65%,5.0,2.0,1.0,2.0,17%,1.65,https://finance.naver.com/sise/sise_group_deta...,"[미스터블루 *, 디앤씨미디어 *, 한류타임즈 *, 디지틀조선 *, 아시아경제 *]"
5,문구류,+1.57%,3.0,1.0,0.0,2.0,16%,1.57,https://finance.naver.com/sise/sise_group_deta...,"[삼성출판사, 양지사 *, 모나미]"
6,방송과엔터테인먼트,+1.47%,52.0,29.0,5.0,18.0,15%,1.47,https://finance.naver.com/sise/sise_group_deta...,"[SBS, 와이지엔터테인먼트 *, CJ CGV, 갤럭시아에스엠, NEW *, 제이웨..."
7,컴퓨터와주변기기,+1.39%,18.0,8.0,1.0,9.0,14%,1.39,https://finance.naver.com/sise/sise_group_deta...,"[피씨디렉트 *, 주연테크, 청호컴넷, 푸른기술 *, 한국정보공학 *, 에이텍 *,..."
8,반도체와반도체장비,+1.35%,112.0,74.0,2.0,36.0,14%,1.35,https://finance.naver.com/sise/sise_group_deta...,"[에스앤에스텍 *, 테크윙 *, 에이디테크놀로지 *, SFA반도체 *, 램테크놀러지..."
9,화학,+1.28%,110.0,42.0,7.0,61.0,13%,1.28,https://finance.naver.com/sise/sise_group_deta...,"[에코프로 *, 코스모화학, 진양폴리, 코스모신소재, LG화학, SKC, 원익머트리..."


In [6]:
def getThemeDataPages(pageNums):
    themes = pd.DataFrame()
    linkDict = {}
    
    for pageNum in pageNums:
        url = f'https://finance.naver.com/sise/theme.nhn?&page={pageNum}'
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        
        df = pd.read_html(html)[0].reset_index(drop=True)
        df.columns = [i[1] for i in df.columns]
        df = df[df.테마명.isna() == False]
        themes = pd.concat([themes, df])
        
        # findLink
        for link in soup.findAll("a", href=re.compile("^(/sise/sise_group_detail.nhn)((?!:).)*$")):
            if 'href' in link.attrs:
                    linkDict[link.text] = 'https://finance.naver.com' + link.attrs['href']
        
    themes = themes.reset_index(drop=True)
    themes['링크'] = themes.테마명.apply(lambda x: linkDict.get(x))
    
    return themes

In [7]:
def getThemeData():
    url = 'https://finance.naver.com/sise/theme.nhn?&page=1'
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')

    pageNums = pd.read_html(html)[1].columns[1:]
    
    df = getThemeDataPages(pageNums)
    df['종목'] = df.링크.apply(lambda x: getNames(x))
    summary = pd.concat([df.head(10), df.tail(10)])
    return summary, df

In [8]:
summary, df = getThemeData()

In [9]:
df[df.테마명 == '황사/미세먼지'].종목.tolist()

[['하츠 *',
  '상아프론테크 *',
  '파세코 *',
  '포스코 ICT *',
  '보령제약',
  '위닉스 *',
  '휴비츠 *',
  '안국약품 *',
  '한국테크놀로지 *',
  '누리플랜 *',
  '웅진코웨이',
  '삼일제약',
  'KC코트렐',
  '디에이치피코리아 *',
  '비디아이 *',
  '락앤락',
  '위니아딤채 *',
  'JW중외제약',
  '나노 *',
  '성창오토텍 *',
  '롯데하이마트',
  '크린앤사이언스 *',
  '에스디생명공학 *',
  '오공 *',
  '웰크론 *',
  '국제약품',
  '케이엠 *',
  '모나리자']]

In [10]:
summary

,테마명,전일대비,최근3일등락률(평균),상승,보합,하락,주도주,주도주.1,링크,종목
0,육계,+9.40%,-2.08%,8.0,0.0,0.0,마니커에프..,체리부로,https://finance.naver.com/sise/sise_group_deta...,"[마니커에프앤지 *, 체리부로 *, 마니커, 푸드나무 *, 하림 *, 동우팜투테이블..."
1,수산,+5.93%,-1.28%,6.0,2.0,2.0,신라에스지,CJ씨푸드,https://finance.naver.com/sise/sise_group_deta...,"[신라에스지 *, CJ씨푸드, 동원수산, 사조오양, 한성기업, 사조씨푸드, 신라교역..."
2,구제역/광우병 수혜,+4.96%,-2.36%,15.0,2.0,3.0,신라에스지,CJ씨푸드,https://finance.naver.com/sise/sise_group_deta...,"[신라에스지 *, CJ씨푸드, 마니커, 동원수산, 우성사료, 사조오양, 하림 *, ..."
3,"스포츠행사 수혜(올림픽, 월드컵 등)",+3.83%,-1.28%,10.0,0.0,1.0,마니커,아프리카T..,https://finance.naver.com/sise/sise_group_deta...,"[마니커, 아프리카TV *, SBS, 하림 *, 동우팜투테이블 *, SBS콘텐츠허브..."
4,화학섬유,+3.11%,+0.04%,4.0,0.0,4.0,휴비스,효성티앤씨,https://finance.naver.com/sise/sise_group_deta...,"[휴비스, 효성티앤씨, 성안, 대한화섬, 티케이케미칼 *, 태광산업, 코오롱머티리얼..."
5,그래핀,+3.05%,-1.87%,9.0,1.0,3.0,나노메딕스,국일제지,https://finance.naver.com/sise/sise_group_deta...,"[나노메딕스, 국일제지 *, 엘엠에스 *, 크리스탈신소재 *, 엑사이엔씨 *, 덕산..."
6,골판지 제조,+3.02%,+0.09%,7.0,0.0,5.0,대림제지,태림포장,https://finance.naver.com/sise/sise_group_deta...,"[대림제지 *, 태림포장, 대양제지 *, 영풍제지, 삼보판지 *, 신대양제지, 수출..."
7,반도체 대표주(생산),+2.67%,-1.71%,3.0,0.0,0.0,DB하이텍,삼성전자,https://finance.naver.com/sise/sise_group_deta...,"[DB하이텍, 삼성전자, SK하이닉스]"
8,SSD,+2.65%,-1.12%,7.0,0.0,1.0,SFA반도..,심텍,https://finance.naver.com/sise/sise_group_deta...,"[SFA반도체 *, 심텍 *, 하나마이크론 *, KMH하이텍 *, 삼성전자, SK하..."
9,3D 낸드(NAND),+2.55%,-1.52%,17.0,1.0,2.0,테크윙,심텍,https://finance.naver.com/sise/sise_group_deta...,"[테크윙 *, 심텍 *, 피에스케이 *, 디엔에프 *, 유니테스트 *, 테스 *, ..."


In [14]:
상장법인목록 = pd.read_excel('./상장법인목록/상장법인목록.xlsx')

In [93]:
def 종목조회(종목명):
    종목코드 = 상장법인목록[상장법인목록.회사명 == 종목명].종목코드.values[0]
    url = 'https://finance.naver.com/item/sise.nhn?code=' + str(종목코드)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')

    return pd.read_html(html)[0]

In [94]:
종목조회('NAVER')

,0,1,2,3
0,현재가,NaN,매도호가,NaN
1,전일대비,0,매수호가,NaN
2,등락률(%),0.00%,전일가,NaN
3,거래량,NaN,시가,NaN
4,거래대금(천),0,고가,NaN
5,액면가,NaN,저가,NaN
6,NaN,NaN,NaN,NaN
7,상한가,NaN,전일상한,NaN
8,하한가,NaN,전일하한,NaN
9,PER,NaN,EPS,NaN
